# Blog Feedback Dataset - Prediction Intervals using `pitci`
This notebook gives an example of using the `XGBoosterAbsoluteErrorConformalPredictor` and `XGBoosterLeafNodeScaledConformalPredictor` classes to generate non-varying and varying prediction intervals respectively, on the [blog feedback dataset](https://archive.ics.uci.edu/ml/datasets/BlogFeedback).

In [1]:
import pandas as pd
import numpy as np
import xgboost as xgb
import requests
import zipfile
from pathlib import Path
from sklearn.metrics import mean_absolute_error, r2_score, mean_squared_error

In [2]:
import pitci
pitci.__version__

'0.2.0'

# Build example xgboost model

## Download data

In [3]:
zip_address = (
    "https://archive.ics.uci.edu/ml/machine-learning-databases/00304/BlogFeedback.zip"
)

In [4]:
dataloaded_data_location = "data/BlogFeedback.zip"

In [5]:
if not Path(dataloaded_data_location).is_file():

    r = requests.get(zip_address)

    with open(dataloaded_data_location, "wb") as f:

        f.write(r.content)

    with zipfile.ZipFile(dataloaded_data_location, "r") as zip_ref:

        zip_ref.extractall("data/blogfeedback")

## Import data

In [6]:
train = pd.read_csv("data/blogfeedback/blogData_train.csv", header=None)
train.rename(columns={280: "number_comments"}, inplace=True)

In [7]:
test = pd.concat(
    [pd.read_csv(x, header=None) for x in Path("data/blogfeedback/").glob("*test*")],
    axis=0,
)
test.rename(columns={280: "number_comments"}, inplace=True)

## Add sample column
Create a sample column with 3 values that will be used in the following way; <br>
1: (65%) train <br>
2: (17.5%) validate (early stopping) <br>
3: (17.5%) interval <br>
The test sample is provided in a different dataframe.

In [8]:
np.random.seed(1)
random_col = np.random.random(train.shape[0])
train["sample"] = "train"
train.loc[random_col > 0.65, "sample"] = "validate"
train.loc[random_col > 0.825, "sample"] = "interval"

## Append train and test datasets

In [9]:
test["sample"] = "test"

In [10]:
train = train.append(test)

## Create xgboost DMatrices

In [11]:
response = "number_comments"

In [12]:
xgb_data_train = xgb.DMatrix(
    data=train.loc[train["sample"] == "train"].drop(columns=[response, "sample"]),
    label=train.loc[train["sample"] == "train", response],
)

In [13]:
xgb_data_valid = xgb.DMatrix(
    data=train.loc[train["sample"] == "validate"].drop(columns=[response, "sample"]),
    label=train.loc[train["sample"] == "validate", response],
)

In [14]:
xgb_data_interval = xgb.DMatrix(
    data=train.loc[train["sample"] == "interval"].drop(columns=[response, "sample"]),
    label=train.loc[train["sample"] == "interval", response],
)

In [15]:
xgb_data_test = xgb.DMatrix(
    data=train.loc[train["sample"] == "test"].drop(columns=[response, "sample"]),
    label=train.loc[train["sample"] == "test", response],
)

## Build model

In [16]:
model = xgb.train(
    params={"max_depth": 5, "eta": 0.05},
    dtrain=xgb_data_train,
    num_boost_round=500,
    evals=[(xgb_data_valid, "valid")],
    early_stopping_rounds=5,
)

[0]	valid-rmse:36.10511
[1]	valid-rmse:35.12481
[2]	valid-rmse:34.19807
[3]	valid-rmse:33.35577
[4]	valid-rmse:32.54045
[5]	valid-rmse:31.77661
[6]	valid-rmse:31.07890
[7]	valid-rmse:30.47846
[8]	valid-rmse:29.92681
[9]	valid-rmse:29.48078
[10]	valid-rmse:29.02127
[11]	valid-rmse:28.59313
[12]	valid-rmse:28.22241
[13]	valid-rmse:27.85019
[14]	valid-rmse:27.52244
[15]	valid-rmse:27.21372
[16]	valid-rmse:26.93135
[17]	valid-rmse:26.71505
[18]	valid-rmse:26.53474
[19]	valid-rmse:26.35969
[20]	valid-rmse:26.16496
[21]	valid-rmse:25.99391
[22]	valid-rmse:25.88487
[23]	valid-rmse:25.75129
[24]	valid-rmse:25.61705
[25]	valid-rmse:25.53448
[26]	valid-rmse:25.43436
[27]	valid-rmse:25.32747
[28]	valid-rmse:25.23336
[29]	valid-rmse:25.22537
[30]	valid-rmse:25.19106
[31]	valid-rmse:25.17925
[32]	valid-rmse:25.15325
[33]	valid-rmse:25.12523
[34]	valid-rmse:25.06228
[35]	valid-rmse:25.01829
[36]	valid-rmse:24.95198
[37]	valid-rmse:24.98485
[38]	valid-rmse:24.92426
[39]	valid-rmse:24.88181
[40]	valid

# Generate prediction intervals

## Non-varying intervals with `AbsoluteErrorConformalPredictor`
### Calibrate conformal predictor

In [17]:
confo_model1 = pitci.get_absolute_error_conformal_predictor(model)
type(confo_model1)

pitci.xgboost.XGBoosterAbsoluteErrorConformalPredictor

In [18]:
confo_model1.calibrate(data=xgb_data_interval, alpha=0.8)

In [19]:
confo_model1.baseline_interval

3.4589214

### Generate prediction intervals

In [20]:
pred_intervals = confo_model1.predict_with_interval(xgb_data_interval)

In [21]:
pitci.helpers.check_response_within_interval(
    intervals_with_predictions=pred_intervals,
    response=train.loc[train["sample"] == "interval", response],
)

True     0.800066
False    0.199934
Name: number_comments, dtype: float64

In [22]:
pitci.helpers.check_interval_width(intervals_with_predictions=pred_intervals)

0.0     6.917839e+00
0.05    6.917843e+00
0.1     6.917843e+00
0.2     6.917843e+00
0.3     6.917843e+00
0.4     6.917843e+00
0.5     6.917843e+00
0.6     6.917843e+00
0.7     6.917843e+00
0.8     6.917843e+00
0.9     6.917843e+00
0.95    6.917843e+00
1.0     6.917847e+00
mean    6.917844e+00
std     6.872427e-07
iqr     0.000000e+00
dtype: float64

## Varying intervals with `LeafNodeScaledConformalPredictor`
### Calibrate conformal predictor

In [23]:
confo_model2 = pitci.get_leaf_node_scaled_conformal_predictor(model)
type(confo_model2)

pitci.xgboost.XGBoosterLeafNodeScaledConformalPredictor

In [24]:
confo_model2.calibrate(data=xgb_data_interval, alpha=0.8)

In [25]:
confo_model2.baseline_interval

349604.1568684578

### Generate interval sample predictions

In [26]:
pred_intervals = confo_model2.predict_with_interval(xgb_data_interval)

In [27]:
pitci.helpers.check_response_within_interval(
    intervals_with_predictions=pred_intervals,
    response=train.loc[train["sample"] == "interval", response],
)

True     0.797008
False    0.202992
Name: number_comments, dtype: float64

In [28]:
pitci.helpers.check_interval_width(intervals_with_predictions=pred_intervals)

0.0        2.605137
0.05       2.605137
0.1        2.605137
0.2        2.605137
0.3        2.687857
0.4        2.792098
0.5        3.065775
0.6        3.702826
0.7        4.405408
0.8        5.927552
0.9       13.830570
0.95      31.193855
1.0     5549.272331
mean      17.495547
std      122.076161
iqr        2.101584
dtype: float64

In [29]:
pred_intervals = pitci.helpers.prepare_prediction_interval_df(
    pred_intervals, train.loc[train["sample"] == "interval", response]
)

In [30]:
pred_intervals = pitci.helpers.create_interval_buckets(
    pred_intervals, q=5, duplicates="drop"
)

In [31]:
pred_intervals.groupby("interval_width_bucket").apply(
    lambda x: mean_absolute_error(x["response"], x["prediction"])
)

interval_width_bucket
(2.604, 2.792]        0.417753
(2.792, 3.703]        1.219729
(3.703, 5.928]        3.137185
(5.928, 5549.272]    21.024222
dtype: float64

### Generate test sample predictions

In [32]:
pred_test = confo_model2.predict_with_interval(xgb_data_test)

In [33]:
pred_test = pitci.helpers.prepare_prediction_interval_df(
    pred_test, train.loc[train["sample"] == "test", response]
)

In [34]:
pred_test = pitci.helpers.create_interval_buckets(pred_test, q=5, duplicates="drop")

In [35]:
pred_test.groupby("interval_width_bucket").apply(
    lambda x: mean_absolute_error(x["prediction"], x["response"])
)

interval_width_bucket
(2.604, 2.728]        0.339336
(2.728, 3.477]        1.048279
(3.477, 5.006]        2.569561
(5.006, 3699.515]    19.768743
dtype: float64